-----------
## In this notebook:
* Filter out BRON rows
* Create testing/training/validation DFs
* Copy images into respective folders
-----------

In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import glob
import multiprocessing
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import warnings
warnings.filterwarnings("ignore")

## Load Preicp Dataset

In [2]:
night_precip_df = pd.read_pickle("../DFs/night_precip_df.pkl")

In [14]:
night_precip_df

,index,station,time_5M,tair,ta9m,precip,precip_total,precip_max_intensity,snow_depth,precip_diff,img_path,is_night
17,0,b'SCHU',2015-08-16 01:15:00,19.968010,20.122700,0.157000,28.165001,0.000,NaN,0.157000,/tf/NYSM/archive/nysm/cam_photos/2015/08/16/SC...,True
18,1,b'SCHU',2015-08-21 07:00:00,20.588341,20.579670,3.726999,31.892000,0.000,NaN,0.157000,/tf/NYSM/archive/nysm/cam_photos/2015/08/21/SC...,True
27,2,b'SCHU',2015-08-25 01:15:00,23.688120,24.467199,0.348000,0.000000,0.254,NaN,0.348000,/tf/NYSM/archive/nysm/cam_photos/2015/08/25/SC...,True
28,3,b'SCHU',2015-08-25 01:20:00,22.803360,23.695379,0.373000,0.348000,0.000,NaN,0.025000,/tf/NYSM/archive/nysm/cam_photos/2015/08/25/SC...,True
29,4,b'SCHU',2015-08-25 04:00:00,21.525129,21.797680,2.733000,0.373000,0.748,NaN,2.360000,/tf/NYSM/archive/nysm/cam_photos/2015/08/25/SC...,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2073180,388356,b'YORK',2020-12-26 07:00:00,-5.895573,-6.050005,0.230003,81.180000,0.000,-0.006263,0.070000,/tf/NYSM/archive/nysm/cam_photos/2020/12/26/YO...,True
2073181,388357,b'YORK',2020-12-28 11:00:00,1.457623,1.492350,0.840004,82.279999,0.000,0.000236,0.050003,/tf/NYSM/archive/nysm/cam_photos/2020/12/28/YO...,True
2073183,388358,b'YORK',2020-12-30 22:20:00,4.348372,4.467166,0.299995,82.440002,0.000,-0.004136,0.189995,/tf/NYSM/archive/nysm/cam_photos/2020/12/30/YO...,True
2073184,388359,b'YORK',2020-12-30 22:45:00,4.160859,4.248825,0.409996,82.629997,0.000,-0.003529,0.110001,/tf/NYSM/archive/nysm/cam_photos/2020/12/30/YO...,True


In [24]:
np.unique(night_precip_df['station'].values)

array([b'ADDI', b'ANDE', b'BATA', b'BEAC', b'BELD', b'BELL', b'BELM',
       b'BERK', b'BING', b'BKLN', b'BRAN', b'BREW', b'BROC', b'BRON',
       b'BROO', b'BSPA', b'BUFF', b'BURD', b'BURT', b'CAMD', b'CAPE',
       b'CHAZ', b'CHES', b'CINC', b'CLAR', b'CLIF', b'CLYM', b'COBL',
       b'COHO', b'COLD', b'COPA', b'COPE', b'CROG', b'CSQR', b'DELE',
       b'DEPO', b'DOVE', b'DUAN', b'EAUR', b'EDIN', b'EDWA', b'ELDR',
       b'ELLE', b'ELMI', b'ESSX', b'FAYE', b'FRED', b'GABR', b'GFAL',
       b'GFLD', b'GROT', b'GROV', b'HAMM', b'HARP', b'HARR', b'HART',
       b'HERK', b'HFAL', b'ILAK', b'JOHN', b'JORD', b'KIND', b'LAUR',
       b'LOUI', b'MALO', b'MANH', b'MEDI', b'MEDU', b'MORR', b'NBRA',
       b'NEWC', b'NHUD', b'OLDF', b'OLEA', b'ONTA', b'OPPE', b'OSCE',
       b'OSWE', b'OTIS', b'OWEG', b'PENN', b'PHIL', b'PISE', b'POTS',
       b'RAND', b'RAQU', b'REDF', b'REDH', b'ROXB', b'RUSH', b'SARA',
       b'SBRI', b'SCHA', b'SCHO', b'SCHU', b'SCIP', b'SHER', b'SOME',
       b'SOUT', b'SP

## Create BRON Precip Test/Valid Subset

In [31]:
df_BRON = night_precip_df.loc[night_precip_df['station']==b'BRON']

In [32]:
# change index
df_BRON['index'] = list(range(0,len(df_BRON)))

In [33]:
df_BRON

,index,station,time_5M,tair,ta9m,precip,precip_total,precip_max_intensity,snow_depth,precip_diff,img_path,is_night
879711,0,b'BRON',2018-07-15 08:30:00,21.224859,NaN,20.009998,1.670000,0.92,0.017691,2.530001,/tf/NYSM/archive/nysm/cam_photos/2018/07/15/BR...,True
879827,1,b'BRON',2018-07-22 08:45:00,22.947109,22.928329,18.569996,66.010002,0.00,0.013538,0.049995,/tf/NYSM/archive/nysm/cam_photos/2018/07/22/BR...,True
907357,2,b'BRON',2018-08-22 04:00:00,22.212460,22.226271,0.500000,280.640015,0.00,0.003272,0.129974,/tf/NYSM/archive/nysm/cam_photos/2018/08/22/BR...,True
1105564,3,b'BRON',2018-12-20 23:00:00,5.425528,5.363216,1.439998,1.270000,0.00,0.003449,0.110000,/tf/NYSM/archive/nysm/cam_photos/2018/12/20/BR...,True
1105656,4,b'BRON',2018-12-28 11:00:00,6.468767,6.293943,3.910004,64.489998,0.00,0.007176,0.200005,/tf/NYSM/archive/nysm/cam_photos/2018/12/28/BR...,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2026992,895,b'BRON',2020-12-31 12:10:00,6.191988,6.179770,4.650002,99.730003,0.00,0.013118,0.139999,/tf/NYSM/archive/nysm/cam_photos/2020/12/31/BR...,True
2026993,896,b'BRON',2020-12-31 12:15:00,6.265157,6.267103,4.799995,99.870003,0.00,0.012884,0.149994,/tf/NYSM/archive/nysm/cam_photos/2020/12/31/BR...,True
2026994,897,b'BRON',2020-12-31 12:20:00,6.094457,6.102151,4.949997,100.019997,0.00,0.013430,0.150002,/tf/NYSM/archive/nysm/cam_photos/2020/12/31/BR...,True
2026995,898,b'BRON',2020-12-31 12:25:00,6.169276,6.169355,5.059998,100.169998,0.00,0.013191,0.110001,/tf/NYSM/archive/nysm/cam_photos/2020/12/31/BR...,True


In [49]:
df_BRON_2019 = df_BRON[(df_BRON['time_5M'] < '2020-01-01 00:00:00')]

In [50]:
# create a df with just the images
df_BRON_2019_WP_imgs = df_BRON_2019['img_path']
df_BRON_2019_WP_imgs

879711     /tf/NYSM/archive/nysm/cam_photos/2018/07/15/BR...
879827     /tf/NYSM/archive/nysm/cam_photos/2018/07/22/BR...
907357     /tf/NYSM/archive/nysm/cam_photos/2018/08/22/BR...
1105564    /tf/NYSM/archive/nysm/cam_photos/2018/12/20/BR...
1105656    /tf/NYSM/archive/nysm/cam_photos/2018/12/28/BR...
                                 ...                        
1584707    /tf/NYSM/archive/nysm/cam_photos/2019/12/30/BR...
1584708    /tf/NYSM/archive/nysm/cam_photos/2019/12/30/BR...
1584709    /tf/NYSM/archive/nysm/cam_photos/2019/12/30/BR...
1584710    /tf/NYSM/archive/nysm/cam_photos/2019/12/30/BR...
1584711    /tf/NYSM/archive/nysm/cam_photos/2019/12/30/BR...
Name: img_path, Length: 254, dtype: object

In [51]:
# copy images into no precip folder
destination = "/tf/kokamura/Precip/3st-2cl-gs/WP"
for image in df_BRON_2019_WP_imgs:
    shutil.copy(image, destination)

## Load No Precip DF